## Performance Tests of Coadd Catalog Access
Michael Wood-Vasey
Notebook last run: 2018-07-30

Test the performance of data manipulations of the static coadd catalogs.

### Summary
1. Using a trimmed file consistent only of the columns exposed through the DPDD is 10 times smaller than the full file.
2. Analyses using GCRCatalogs are 10-30 faster with the trimmed files than with the full files.
3. Analyses with the HDF5 `format='table'` are 1-2 times slower than HDF5 `format='fixed'` files.
4. Directly accessing vs. using the GCR iterator had no significant performance impact.
5. A simple Pandas analysis is listed here.
6. A preliminary Dask analyses is conducted here, but a fuller development of Dask will be done in issues #237.

### Discusion
1, 2. We should create trimed file versions for future releases.
3. The `format='table'` was initially mildly surprising.  However, after a more careful reading of the `pytables` documentation this was already predicted:
```
format : {‘fixed’, ‘table’}, default ‘fixed’

    Possible values:

        ‘fixed’: Fixed format. Fast writing/reading. Not-appendable, nor searchable.
        ‘table’: Table format. Write as a PyTables Table structure which may perform worse but allow more flexible operations like searching / selecting subsets of the data.
```

The `format='fixed'` is designed for fast writing/reading.  It's the performance when selecting subsets of data that's potentially faster with the `format='table'`.

The `table` format is also required to use Dask.

### Original Charge
1. Identify trivial, moderate, and worst-case use case examples.
2. Measure performance on
    1. single patch
    2. a single tract
    3. the full dataset
3. Record data sizes of each of the above A, B, C
4. Determine if performance considerations mean we should generate a static file that contains a restricted set in columns.
5. Look into again using full tables functionality to write HDF5 files so that they can be read by column efficiently. This was previously not possible because of an error trying to write the thousands of columns in our full coadd catalogs. This is #158


In [ ]:
import os

import numpy as np

## How fast is GCRCatalogs?

In [2]:
import GCRCatalogs

If you want to use the GCR reader outside of NERSC environment, you can override the `base_dir`.

In [3]:
config = {}

trim_config = config.copy()
trim_config['filename_pattern'] = r'trim_merged_tract_\d+\.hdf5'
table_trim_config = config.copy()
table_trim_config['filename_pattern'] = r'table_trim_merged_tract_\d+\.hdf5'

trim_onetract_config = config.copy()
trim_onetract_config['filename_pattern'] = 'trim_merged_tract_4850.hdf5'
table_trim_onetract_config = config.copy()
table_trim_onetract_config['filename_pattern'] = 'table_trim_merged_tract_4850.hdf5'

### Time loading of GCRCatalogs 

Loading the GCR Catalog is, in principle, just the initialization of the catalog.  In practice the GCRCatalog reader does need to read through all of the metadata in the HDF5 files to figure out what's in there and available.  The onetract version is reading a 7.4 GB file that should fit in memory.  The full Run 1.1p is 78 GB, which does not fit in the average desktop memory.  This size could pontentially fit in the memory of various high-memory shared nodes.  This difference in size is conveniently roughly a factor of 10.  We should naively expect that operations will take 10x longer when all of the data fits in memory, and potentially 10-100x longer when the data do not.  The range is estimated across a variety of usage patterns.

The trim files are 1/10 of the size of the full files due to the removal of 90% of the columns.  The `load_catalog` doesn't load the data, but does need to open and touch each file to read the metadata.  This metadata reading step is only about 2 times faster for the trim files than for the full files.

In [4]:
%%timeit
gc_onetract = GCRCatalogs.load_catalog('dc2_coadd_run1.1p_tract4850', config)

988 ms ± 32.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
gc_onetract_trim = GCRCatalogs.load_catalog('dc2_coadd_run1.1p_tract4850', trim_onetract_config)

560 ms ± 12.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
gc_onetract = GCRCatalogs.load_catalog('dc2_coadd_run1.1p_tract4850', config)
gc_onetract_trim = GCRCatalogs.load_catalog('dc2_coadd_run1.1p_tract4850', trim_onetract_config)
gc_onetract_table_trim = GCRCatalogs.load_catalog('dc2_coadd_run1.1p_tract4850', table_trim_onetract_config)

In [7]:
%%timeit
gc = GCRCatalogs.load_catalog('dc2_coadd_run1.1p', config)

14 s ± 1.78 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
gc_trim = GCRCatalogs.load_catalog('dc2_coadd_run1.1p', trim_config)

7.52 s ± 346 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
gc_table_trim = GCRCatalogs.load_catalog('dc2_coadd_run1.1p', table_trim_config)

14.7 s ± 1.82 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Note above that the table version of the trim catalogs takes the same amount of time as the full file.

In [10]:
# Here we actually run and save the objects for later use.
# In the %%timeit runs above, the resulting bound names and objects are discarded.
gc = GCRCatalogs.load_catalog('dc2_coadd_run1.1p', config)
gc_trim = GCRCatalogs.load_catalog('dc2_coadd_run1.1p', trim_config)
gc_table_trim = GCRCatalogs.load_catalog('dc2_coadd_run1.1p', table_trim_config)

### Time calculation using loaded GCRCatalogs objects

Summary:
  * The trim catalog files are 10-30 faster to process than the full catalog files.
  * The 'table' format trim catalog files take 1-2 as long to process as the trim catalog files.
  * The full catalog files take 330 seconds to compute the average.

In [11]:
def compute_mean_color_slow(catalog):
    """Compute the mean g-r color of all objects in the 'catalog'.
    
    This is a trivial performance case.
    This isn't particularly immediately interesting, but it's a simple arithmetic operation between two columns.
    """
    average_gmr = np.mean(catalog['mag_g'] - catalog['mag_r'])
    return average_gmr

In [12]:
def compute_mean_color_faster(catalog):
    """Compute the mean g-r color of all objects in the 'catalog'.
    
    This is a trivial performance case.
    This isn't particularly immediately interesting, but it's a simple arithmetic operation between two columns.
    """
    data = catalog.get_quantities(['mag_g', 'mag_r'])
    average_gmr = np.mean(data['mag_g'] - data['mag_r'])
    return average_gmr

In [13]:
def compute_mean_color_faster_iter(catalog):
    """Compute the mean g-r color of all objects in the 'catalog' using iterator.
    
    This is a trivial performance case.
    This isn't particularly immediately interesting, but it's a simple arithmetic operation between two columns.
    """
    sum_gmr = count = 0
    for data in catalog.get_quantities(['mag_g', 'mag_r'], return_iterator=True):
        sum_gmr += np.sum(data['mag_g'] - data['mag_r'])
        count += len(data['mag_g'])
    return sum_gmr / count

In [14]:
# def compute_stellar_locus():

We below clear the memory cache with `GCRCatalogs` with `clear_cache()` method on the load object to reset for performance tests.  It's harder to control the underlying caching of the GPFS and kernel filesystem memory.

The average color calculation is 18 times faster with the trim files for one tract.  It's only 2 times faster for the full set of files.

There's no particular difference in elapsed time between the `compute_mean_slow`, `compute_mean_fast` and `compute_mean_fast_iter` function.

In [15]:
%%timeit
gc_onetract.clear_cache()
compute_mean_color_slow(gc_onetract)

18.4 s ± 2.44 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit
gc_onetract_trim.clear_cache()
compute_mean_color_slow(gc_onetract_trim)

853 ms ± 53.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit
gc_onetract_table_trim.clear_cache()
compute_mean_color_slow(gc_onetract_table_trim)

853 ms ± 53.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
gc.clear_cache()
compute_mean_color_slow(gc)

5min 33s ± 5.75 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
gc_onetract.clear_cache()
compute_mean_color_faster(gc_onetract)

18.9 s ± 1.85 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%%timeit
gc_trim.clear_cache()
compute_mean_color_faster(gc_trim)

10.4 s ± 409 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
%%timeit
gc_table_trim.clear_cache()
compute_mean_color_faster(gc_table_trim)

The slowest run took 5.54 times longer than the fastest. This could mean that an intermediate result is being cached.
1min ± 45 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit
gc.clear_cache()
compute_mean_color_faster(gc)

5min 42s ± 26 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit
gc_onetract.clear_cache()
compute_mean_color_faster_iter(gc_onetract)

15.4 s ± 1.36 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit
gc_trim.clear_cache()
compute_mean_color_faster_iter(gc_trim)

11.3 s ± 1.19 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
%%timeit
gc_table_trim.clear_cache()
compute_mean_color_faster_iter(gc_table_trim)

26.3 s ± 2.04 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%%timeit
gc.clear_cache()
compute_mean_color_faster_iter(gc)

5min 28s ± 8.33 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## How fast is Pandas?

In [3]:
import os
import pandas as pd

tract = 4850
datafile_basename = 'merged_tract_%d.hdf5' % tract
datafile_basename_trim = 'trim_' + datafile_basename
datafile_basename_table_trim = 'table_trim_' + datafile_basename

base_dir = '/global/projecta/projectdirs/lsst/global/in2p3/Run1.1/summary'

datafile = os.path.join(base_dir, datafile_basename)
datafile_trim = os.path.join(base_dir, datafile_basename_trim)
datafile_table_trim = os.path.join(base_dir, datafile_basename_table_trim)

key_prefix = 'coadd'
nx, ny = 8, 8
patches = ['%d%d' % (i, j) for i in range(nx) for j in range (ny)]  # Note '%d%d' instead of '%d,%d'
patch = patches[0]
key = '%s_%d_%s' % (key_prefix, tract, patch)

### Time loading of catalog using Pandas

Reading just one patch of the tract.

In [4]:
%%timeit
df = pd.read_hdf(datafile, key=key)

42.5 ms ± 1.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
df_trim = pd.read_hdf(datafile_trim, key=key)

10.3 ms ± 605 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%%timeit
df_table_trim = pd.read_hdf(datafile_table_trim, key=key)

16.5 ms ± 619 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Now we'll load all of the patches in the tract

In [7]:
def load_tract_into_pandas(datafile, tract, key_prefix='coadd'):
    nx, ny = 8, 8
    patches = ['%d%d' % (i, j) for i in range(nx) for j in range (ny)]  # Note '%d%d' instead of '%d,%d'

    dfs = []
    for patch in patches:
        key = '%s_%d_%s' % (key_prefix, tract, patch)
        try:
            df = pd.read_hdf(datafile, key=key)
        except:
            continue
        dfs.append(df)

    df = pd.concat(dfs)
    return df

In [8]:
%%timeit
df_trim = load_tract_into_pandas(datafile_trim, tract=tract)

1.61 s ± 24.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
df_table_trim = load_tract_into_pandas(datafile_table_trim, tract=tract)

2.27 s ± 180 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
df_trim = load_tract_into_pandas(datafile_trim, tract=tract)
df_table_trim = load_tract_into_pandas(datafile_table_trim, tract=tract)

In [12]:
%%timeit
df_trim['g_mag'] - df_trim['r_mag']

1.71 ms ± 244 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%%timeit
df_table_trim['g_mag'] - df_table_trim['r_mag']

347 µs ± 10.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## How fast is Dask?

In [31]:
import dask as da
import dask.dataframe as dd

from dask.distributed import Client
client = Client(processes=False)

In [32]:
tract = 4850

base_dir = '/global/projecta/projectdirs/lsst/global/in2p3/Run1.1/summary'

datafile = os.path.join(base_dir, 'table_trim_merged_tract_%d.hdf5' % tract)
datafile_pattern = os.path.join(base_dir, 'table_trim_merged_tract_*.hdf5')

### Time loading of catalog using Dask

In [33]:
da_df = dd.read_hdf(datafile, key='coadd_*', mode='r')
da_df_all = dd.read_hdf(datafile_pattern, key='coadd_*', mode='r')

In [34]:
df2 = np.mean(da_df['g_mag'] - da_df['r_mag'])
df2_all = np.mean(da_df_all['g_mag'] - da_df_all['r_mag'])

### Time computation using Dask

In [35]:
%%timeit
df2.compute()

2.91 s ± 87.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
%%timeit
df2_all.compute()

1min 3s ± 1.76 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Note that the full DASK calculation takes 20 times longer than the onetract calculation for a data volume only 10-times larger.  We're potentially hitting memory limits, but there also might be some ineficiencies.

DASK takes ~38 seconds to do the color average using the `table_trim_` file, while GCRCatalogs takes ~12 seconds using the `trim_` file.

In [37]:
import os
print(os.getenv('OMP_NUM_THREADS'))

4


### Does it matter if we're on Lustre (SCRATCH)

No.

In [38]:
base_dir = '/global/cscratch1/sd/wmwv/DC2/Run1.1p/summary'

datafile_lustre = os.path.join(base_dir, 'table_trim_merged_tract_%d.hdf5' % tract)
datafile_pattern_lustre = os.path.join(base_dir, 'table_trim_merged_tract_*.hdf5')

In [39]:
da_df_lustre = dd.read_hdf(datafile_lustre, key='coadd_*', mode='r')
da_df_all_lustre = dd.read_hdf(datafile_pattern_lustre, key='coadd_*', mode='r')

In [40]:
df2_lustre = np.mean(da_df_lustre['g_mag'] - da_df_lustre['r_mag'])
df2_all_lustre = np.mean(da_df_all_lustre['g_mag'] - da_df_all_lustre['r_mag'])

In [41]:
%%timeit
df2_lustre.compute()

The slowest run took 5.24 times longer than the fastest. This could mean that an intermediate result is being cached.
5.43 s ± 4.82 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
%%timeit
df2_all_lustre.compute()

1min 28s ± 12.5 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Time on the Luster file system (2, 42) sec seems about the same as the GPFS (2, 50) sec.

In [43]:
import dask
dask.config.set(scheduler='threads')
print(dask.config.config)

{'distributed': {'worker': {'memory': {'target': False, 'spill': False, 'pause': 0.8, 'terminate': 0.95}, 'multiprocessing-method': 'forkserver', 'use-file-locking': True, 'profile': {'interval': '10ms', 'cycle': '1000ms'}}, 'version': 2, 'scheduler': {'allowed-failures': 3, 'bandwidth': 100000000, 'default-data-size': 1000, 'transition-log-length': 100000, 'work-stealing': True, 'worker-ttl': None}, 'client': {'heartbeat': '5s'}, 'comm': {'compression': 'auto', 'default-scheme': 'tcp', 'socket-backlog': 2048, 'recent-messages-log-length': 0, 'timeouts': {'connect': '10s', 'tcp': '30s'}}, 'dashboard': {'link': 'http://{host}:{port}/status', 'export-tool': False}, 'admin': {'tick': {'interval': '20ms', 'limit': '3s'}, 'log-length': 10000, 'log-format': '%(name)s - %(levelname)s - %(message)s', 'pdb-on-err': False}}, 'jobqueue': {'slurm': {'cores': 64, 'memory': '128GB', 'processes': 2, 'queue': 'debug', 'walltime': '00:10:00', 'job-extra': ['-C haswell', '-L SCRATCH, cscratch1']}}, 'arr

In [44]:
%%timeit
df2_lustre.compute()

3.09 s ± 662 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Does it matter if we're on the burst buffer?